In [1]:
import sys
sys.path.extend(['D:\\Documents\\GitHub\\pq-analyzer\\env', 'D:\\Documents\\GitHub\\pq-analyzer\\env\\lib\\site-packages'])

import pandas as pd
import numpy as np
import pickle
from ast import literal_eval

from pdb import set_trace as st
import string
import re
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stop_words = stopwords.words('english')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

from IPython.core.interactiveshell import InteractiveShell
import sys
InteractiveShell.ast_node_interactivity = 'all'
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
np.set_printoptions(threshold = sys.maxsize)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\thisi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\thisi\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thisi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('../pq-scraper/pqs.csv', sep='|')
for key in ['asker_parliaments', 'askees', 'askees_portfolios', 'responses']:
    df[key] = df[key].apply(literal_eval)
print('Number of rows:', len(df))
assert isinstance(df.loc[0].responses, list)
df.dtypes
#df.head(2)

Number of rows: 19801


asker_name           object
asker_party          object
asker_parliaments    object
askees               object
askees_portfolios    object
question             object
parliament_no         int64
session_no            int64
volume_no             int64
sitting_no            int64
sitting_date         object
report_section       object
title                object
responses            object
is_pq                  bool
dtype: object

asker_name                     asker_party asker_parliaments  \
0      Lee Bee Wah           People's Action Party      (11, 12, 13)   
1  K Thanaletchimi  Nominated Member of Parliament             (13,)   

                                                                                                                 askees  \
0  (Senior Minister of State for the Environment and Water Resources, Minister for the Environment and Water Resources)   
1  (Senior Minister of State for the Environment and Water Resources, Minister for the Environment and Water Resources)   

  askees_portfolios  \
0            (MSE,)   
1            (MSE,)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [3]:
gin_stopwords = ['ability', 'able', 'about', 'abt', 'actually', 'ah', 'agree', 'already', 'also', 'always',
                 'am', 'art', 'artist', 'because', 'bring', 'can', 'can\'t', 'cannot', 'cant', 'come',
                 'could', 'couldn\'t', 'couldnt', 'did', 'didn\'t', 'didnt', 'different', 'don\'t', 'dont',
                 'eg', 'eh', 'ei', 'etc', 'even', 'every', 'everywhere', 'example', 'feel', 'felt', 'get',
                 'go', 'good', 'ha', 'haha', 'hahaha', 'hahahaha', 'hence', 'i\'m', 'ie', 'im', 'important', 
                 'k', 'know', 'la', 'like', 'make', 'may', 'maybe', 'need', 'often', 'oh', 'ok',
                 'one', 'people', 'put', 'quite', 'rather', 'really', 'say', 'singapore', 'something', 'su', 'space',
                 'still', 'th', 'that', 'that\'s', 'thats', 'they', 'they\'re', 'they\'ve', 'theyre', 'theyve',
                 'think', 'three', 'try', 'tt', 'two', 'u', 'us', 'use', 've', 'wa', 'want', 'way', 'we',
                 'we\'re', 'we\'ve', 'well', 'were', 'weve', 'whatsoever', 'won\'t', 'wont', 'work',
                 'would', 'wouldn\'t', 'wouldnt', 'y\'all', 'yall', 'yea', 'yeah', 'you\'d', 'you\'ll', 
                 'you\'re', 'you\'ve', 'youd', 'youll', 'youre', 'youve', 'yr']

honorific_stopwords = ['mrs', 'mr', 'ms', 'miss', 'mdm', 'madam', 'mister', 'er', 'assoc', 'prof', 'dr', 'bg']

hansard_stopwords = ['speaker', 'leader', 'chairman', 'deputy', 'beg', 'permission', 'withdraw', 'cut', 'thank', 'member', 'honorable', 'question', 'clarification', 'grateful']

add_stopwords = gin_stopwords + honorific_stopwords + hansard_stopwords
stop_words.extend(add_stopwords)

In [4]:
phrases = ['often like']

In [5]:
# Saving stopwords and phrases into files
with open('stopwords.pkl', 'wb') as f:
    pickle.dump(stop_words, f)
    
with open('stopphrases.pkl', 'wb') as f:
    pickle.dump(phrases, f)

## 0. Functions

In [6]:
def clean_text(text, stop_words = stop_words, phrases = phrases, remove_phrases = False):
    lemmatizer = WordNetLemmatizer()
    
    if text != text: # np.nan
        return np.nan
    
    else:
        text = str(text)
        
        # Removing hashtags, mentions, links etc.
        text = [word.lower().strip() for word in text.split(' ') if word != '']
        text = [word for word in text if word != '']
        text = [word for word in text if word[0] != '#']
        text = [word for word in text if word[0] != '@']
        text = [word for word in text if word[0:4] != 'http']
        text = [word for word in text if word[0:3] != 'www']
        text = ' '.join(text)

        # Replacing question marks and exclamation marks with periods
        text = re.sub('[?!]', '.', text)

        # Replacing symbols to avoid erroneously joined words
        text = text.replace('\n ', ' ')
        text = text.replace('\n', ' ')
        text = text.replace('&gt; ', ' ')
        text = text.replace('&gt', ' ')
        text = text.replace('/', ' ')
        text = text.replace('&amp;', '&')
        text = text.replace('-', ' ')
        text = text.replace('\'', '')
        text = ''.join([c for c in list(text) if c in string.ascii_lowercase +  ' \''])
        
        # Replacing certain abbreviations with the full text or otherwise (to prevent removal during stop words removal)
        ## For Word/phrases that would have been removed during lemmatization
        text = ' ' + text + ' '
        text = text.replace(' ses ', 'socioeconomicstatus')
        text = text.strip()

        # Lemmatizing text
        text = [lemmatizer.lemmatize(word, pos = 'a') for word in text.split(' ') if word != '']
        text = [lemmatizer.lemmatize(word, pos = 'n') for word in text]
        text = [lemmatizer.lemmatize(word, pos = 'v') for word in text]
        text = [lemmatizer.lemmatize(word, pos = 'r') for word in text]
        text = [lemmatizer.lemmatize(word, pos = 's') for word in text]
        text = ' '.join(text)
        
        # Replacing certain abbreviations with the full text or otherwise (to prevent removal during stop words removal)
        text = ' ' + text + ' '
        text = text.replace(' ad hoc ', ' adhoc ')
        text = text.replace(' arh ', ' arts resource hub ')
        text = text.replace(' cd ', ' capability development ')
        text = text.replace(' coy ', ' company ')
        text = text.replace(' fb ', ' food beverage ')
        text = text.replace(' gov ', ' government ')
        text = text.replace(' ip ', ' intellectual property ')
        text = text.replace(' national art council ', ' nac ')
        text = text.replace(' otg ', ' organisation transformation grant ')
        text = text.replace(' ppl ', ' people ')
        text = text.replace(' sg ', ' singapore ')
        text = text.replace(' sifa ', ' singapore international festival of art ')
        text = text.replace(' swf ', ' singapore writer festival ')
        text = text.strip()

        # Removing phrases
        if remove_phrases == True:
            for phrase in phrases:
                text = text.replace(phrase, '')

        # Removing stop words
        text = ' '.join([word.strip() for word in text.split(' ') if word.strip() not in stop_words])

        text = text.replace('   ', ' ')
        text = text.replace('  ', ' ')

        return text

In [7]:
def str_replace(string, to_replace, replace_value):
    if string != string: # np.nan
        return np.nan
    
    else:
        return string.replace(to_replace, replace_value).strip()

In [8]:
def num_words(string):
    if string != string: # np.nan
        return np.nan
    
    else: 
        words = [word for word in string.split(' ') if word != '']
        
        return len(words)

In [24]:
def obtain_vocab(col, max_df = 0.5, min_df = 0.001, stop_words = stop_words):
    
    col = col.dropna()
    
    vectorizer = TfidfVectorizer(max_df = max_df,
                                 min_df = min_df,
                                 max_features = None,
                                 ngram_range = (1,1), # avoid memory issues
                                 strip_accents = 'ascii',
                                 stop_words = stop_words)
    
    t = vectorizer.fit_transform(col)
    # Version Issue: get_feature_names_out() may not work; try get_feature_names() instead if so
    if 'get_feature_names_out' in dir(vectorizer):
        ngram = vectorizer.get_feature_names_out()
    else:
        ngram = vectorizer.get_feature_names()

    name_vec_df = pd.DataFrame(t.toarray(), columns = ngram)
    ngram_freq = name_vec_df.sum(axis = 0).reset_index()
    ngram_freq.columns = ['word/phrase', 'score']
    
    return ngram_freq

## 1. Preprocessing Text

In [10]:
# Cleaning topics
#df['Topic'] = df['Topic'].apply(str_replace, args = ('\xa0', ''))

#df.loc[df['Topic'].str.contains('Concerted IR strategy', na = False), 'Topic'] = '[Internationalisation- Possible work areas] Concerted IR strategy for key markets, including the cultivation of promising artists and groups for profiling. Engaging SG artists who have strong links abroad to bring along other artists to these platforms and hunting in packs'
#df.loc[df['Topic'].str.contains('NAC\'s presence in key markets', na = False), 'Topic'] = '[Internationalisation- Possible work areas] Establish NAC\'s presence in key markets, either through partnering Singapore offices or establishing Cultural Office(s) abroad'
#df.loc[df['Topic'].str.contains('in the region to strengthen Singapore\'s position', na = False), 'Topic'] = '[Internationalisation- Possible work areas] Identify & establish thought leadership platforms in the region to strengthen Singapore\'s position as hub to SEA'

In [11]:
qn_raw_col = 'question'
res_raw_col = 'responses'

qn_col = 'question_cleaned'
res_col = 'responses_cleaned'

In [12]:
df.insert(loc = 6,
          column = qn_col,
          value = df[qn_raw_col].apply(clean_text))

In [13]:
df.insert(loc = 15,
          column = res_col,
          value = df[res_raw_col].apply(lambda responses: list(map(lambda r: (r[0], r[1], r[2], clean_text(r[3])), responses))))

In [14]:
df['question_len'] = df[qn_raw_col].apply(num_words)
df['response_count'] = df[res_raw_col].apply(len)
df['response_total_len'] = df[res_raw_col].apply(lambda x: sum(list(map(lambda y: num_words(y[3]), x))))

In [15]:
#df.head(2)

asker_name                     asker_party asker_parliaments  \
0      Lee Bee Wah           People's Action Party      (11, 12, 13)   
1  K Thanaletchimi  Nominated Member of Parliament             (13,)   

                                                                                                                 askees  \
0  (Senior Minister of State for the Environment and Water Resources, Minister for the Environment and Water Resources)   
1  (Senior Minister of State for the Environment and Water Resources, Minister for the Environment and Water Resources)   

  askees_portfolios  \
0            (MSE,)   
1            (MSE,)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [16]:
df.to_csv('pqs_processed.csv', index = False)

In [26]:
# Obtaining vocabulary for feedback and recommendations
## This gives us a sense of other stopwords we may have missed, or abbreviations that should be expanded. 
qn_vocab = obtain_vocab(df[qn_col], min_df = 0.0001)
res_vocab = obtain_vocab(df[res_col].apply(lambda responses: ' '.join(list(map(lambda response: response[3], responses)))), min_df = 0.0001)

qn_vocab.head(2)
res_vocab.head(2)

qn_vocab.to_csv('qn_vocab.csv', index = False)
res_vocab.to_csv('res_vocab.csv', index = False)

,word/phrase,score
0,aaa,0.273082
1,aar,2.171879


,word/phrase,score
0,aa,3.215829
1,aaa,1.686813


In [27]:
qn_vocab['len'] = qn_vocab['word/phrase'].apply(len)
res_vocab['len'] = res_vocab['word/phrase'].apply(len)

qn_vocab[qn_vocab['len'] <= 3]
res_vocab[qn_vocab['len'] <= 3]

,word/phrase,score,len
0,aaa,0.273082,3
1,aar,2.171879,3
2,ab,1.525495,2
8,abc,3.777815,3
14,abe,0.213949,3
50,abu,0.445136,3
58,ac,0.255916,2
112,ace,2.277386,3
122,acp,0.665657,3
137,act,99.699557,3


<ipython-input-27-c89e604dad07>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  res_vocab[qn_vocab['len'] <= 3]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [28]:
# Searching for qns/responses for which cleaned version contains certain words/phrases
string = 'everywhere'
df[[qn_raw_col, qn_col]][df[qn_col].str.contains(string, na = False)]
df[[res_raw_col, res_col]][df[res_col].str.contains(string, na = False)]

,question,question_cleaned


,responses,responses_cleaned
